13.2.5

Consider four possible models for predicting house prices:

Using only the size and number of rooms.

Using size, number of rooms, and building type.

Using size and building type, and their interaction.

Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Set up a pipeline for each of these four models.

Then, get predictions on the test set for each of your pipelines, and compute the root mean squared error. Which model performed best?

Note: You should only use the function train_test_split() one time in your code; that is, we should be predicting on the same test set for all three models

In [82]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [83]:
df = pd.read_csv("AmesHousing.csv")
df = df.rename(columns={"SalePrice":"saleprice", "Gr Liv Area":"size", "TotRms AbvGrd":"totalrooms", "Bldg Type":"buildingtype"})
df.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,saleprice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [84]:
X = df[["size", "totalrooms", "buildingtype"]]
y = df["saleprice"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

In [85]:
model_1_pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ["size", "totalrooms"])
        ])),
    ('regressor', LinearRegression())
]).set_output(transform="pandas")

In [86]:
model_2_pipeline = Pipeline(steps=[
    ('preprocessor', ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), ["size", "totalrooms"]),
            ('cat', OneHotEncoder(drop="first"), ["buildingtype"])
        ])),
    ('regressor', LinearRegression())
])#.set_output(transform="pandas")

In [87]:
ct_dummies = ColumnTransformer(
  [("dummify", OneHotEncoder(sparse_output = False), ["buildingtype"])],
  remainder = "passthrough").set_output(transform = "pandas")

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["remainder__totalrooms", "dummify__buildingtype_1Fam"]),
  ],
  remainder = "drop").set_output(transform = "pandas")

model_3_pipeline = Pipeline(steps=[
    ('dummy', ct_dummies),
    ("interaction", ct_inter),
    ('regressor', LinearRegression())
])#.set_output(transform="pandas")

In [88]:
ct_poly = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["buildingtype"]),
    ("polynomial", PolynomialFeatures(degree=5, include_bias=False), ["size", "totalrooms"])
  ],
  remainder = "drop"
)

model_4_pipeline = Pipeline(steps=[
    ("preprocessing", ct_poly),
    ("linear_regression", LinearRegression())
])#.set_output(transform="pandas")

In [89]:
#predictions

model_1_pipeline.fit(X_train, y_train)
y_pred1 = model_1_pipeline.predict(X_test)
rmse1 = np.sqrt(mean_squared_error(y_test, y_pred1))
print("RMSE for model 1:", rmse1)
print()

model_2_pipeline.fit(X_train, y_train)
y_pred2 = model_2_pipeline.predict(X_test)
rmse2 = np.sqrt(mean_squared_error(y_test, y_pred2))
print("RMSE for model 2:", rmse2)
print()

model_3_pipeline.fit(X_train, y_train)
y_pred3 = model_3_pipeline.predict(X_test)
rmse3 = np.sqrt(mean_squared_error(y_test, y_pred3))
print("RMSE for model 3:", rmse3)
print()

model_4_pipeline.fit(X_train, y_train)
y_pred4 = model_4_pipeline.predict(X_test)
rmse4 = np.sqrt(mean_squared_error(y_test, y_pred4))
print("RMSE for model 4:", rmse4)

RMSE for model 1: 57845.470405394

RMSE for model 2: 55283.65307521667

RMSE for model 3: 64201.31955414996

RMSE for model 4: 137882.7527187992


Model 2, where saleprice is on size, buildingtype, and totalrooms, performed best as it has the lowest RMSE.

Once again consider four modeling options for house price:

Using only the size and number of rooms.

Using size, number of rooms, and building type.

Using size and building type, and their interaction.

Using a 5-degree polynomial on size, a 5-degree polynomial on number of rooms, and also building type.

Use cross_val_score with the pipelines you made earlier to find the cross-validated root mean squared error for each model.

Which do you prefer? Does this agree with your conclusion from earlier?

In [90]:
scores1 = cross_val_score(model_1_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
scores2 = cross_val_score(model_2_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
scores3 = cross_val_score(model_3_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
scores4 = cross_val_score(model_4_pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
scores1.mean(), scores2.mean(), scores3.mean(), scores4.mean()

(-3136138908.1709027,
 -2951993958.1007304,
 -4258019084.8761864,
 -3627125054.898018)

From the cross validated RMSE, the second model performs better than the other ones because it has the lowest RMSE. This agrees with what we concluded in the first problem.

Consider one hundred modeling options for house price:

House size, trying degrees 1 through 10

Number of rooms, trying degrees 1 through 10

Building Type

Hint: The dictionary of possible values that you make to give to GridSearchCV will have two elements instead of one.

Q1: Which model performed the best?

Q2: What downsides do you see of trying all possible model options? How might you go about choosing a smaller number of tuning values to try?

In [105]:
ct_poly2 = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["buildingtype"]),
    ("polynomial", PolynomialFeatures(include_bias=False), ["size", "totalrooms"])
  ],
  remainder = "drop"
)

lr_pipeline_poly = Pipeline(
  [("preprocessing", ct_poly2),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {'preprocessing__polynomial__degree': np.arange(1, 11)}

gscv = GridSearchCV(lr_pipeline_poly, degrees, cv = 5, scoring='neg_mean_squared_error')
gscv_fitted = gscv.fit(X, y)

best_params = gscv.best_params_
best_rmse = np.sqrt(-gscv.best_score_)
print("Best parameters:", best_params)
print("Best RMSE:", best_rmse)

Best parameters: {'preprocessing__polynomial__degree': 3}
Best RMSE: 53935.15246151222


The model with polynomial degree 3 is the best model with lowest RMSE.